In [45]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Clone TensorFlow Object Detection API
!git clone https://github.com/tensorflow/models.git

# Install dependencies
!pip install tf-slim
!pip install tensorflow-object-detection-api


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Cloning into 'models'...
remote: Enumerating objects: 98993, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 98993 (delta 12), reused 6 (delta 5), pack-reused 98973 (from 2)
Receiving objects: 100% (98993/98993), 622.73 MiB | 16.32 MiB/s, done.
Resolving deltas: 100% (71981/71981), done.


In [46]:
import os
import tensorflow as tf
from object_detection.utils import dataset_util

# Path dataset
DATASET_PATH = '/content/drive/My Drive/dataset/kupu'

# Fungsi untuk membuat TFRecord
def create_tf_record(output_file, image_dir, annotations):
    writer = tf.io.TFRecordWriter(output_file)

    # Loop over semua gambar
    for image_filename in os.listdir(image_dir):
        # Path lengkap gambar dan label
        image_path = os.path.join(image_dir, image_filename)
        label = annotations.get(image_filename, [])

        # Muat gambar dan konversi ke byte
        with tf.io.gfile.GFile(image_path, 'rb') as fid:
            encoded_image = fid.read()

        # Buat contoh TFRecord
        tf_example = dataset_util.create_tf_example(
            filename=image_filename,
            encoded_image=encoded_image,
            annotations=label
        )
        writer.write(tf_example.SerializeToString())

    writer.close()

# Contoh konversi dataset
create_tf_record(
    output_file='/content/drive/My Drive/kupu_train.record',
    image_dir=f'{DATASET_PATH}/train',
    annotations={}
)
create_tf_record(
    output_file='/content/drive/My Drive/kupu_val.record',
    image_dir=f'{DATASET_PATH}/val',
    annotations={}
)


In [47]:
# Unduh model Faster R-CNN
!wget http://download.tensorflow.org/models/object_detection/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz
!tar -xvf faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz


--2024-12-21 02:16:04--  http://download.tensorflow.org/models/object_detection/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.251.175.207, 74.125.24.207, 142.251.10.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|142.251.175.207|:80... connected.
HTTP request sent, awaiting response... 403 Forbidden
2024-12-21 02:16:04 ERROR 403: Forbidden.

tar: faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [48]:
!cp faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/pipeline.config /content/drive/My Drive/


cp: target 'Drive/' is not a directory


In [49]:
# Jalankan pelatihan
!python models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path=/content/drive/My Drive/pipeline.config \
    --model_dir=/content/drive/My Drive/faster_rcnn_model \
    --num_train_steps=20000 \
    --sample_1_of_n_eval_examples=1 \
    --alsologtostderr


2024-12-21 02:16:17.465595: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-21 02:16:17.487368: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-21 02:16:17.493573: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-21 02:16:19.279599: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Traceback (most recent call last):
  File "/content/models/research/object_detection/model_main_tf2.py", line 31, in <module>
    from object_detection import model_lib_v2
ImportError: cannot import name 'model_lib_v2' from 'object_detection

In [50]:
%load_ext tensorboard
%tensorboard --logdir /content/drive/My Drive/faster_rcnn_model


ERROR: Failed to launch TensorBoard (exited with 2).
Contents of stderr:
2024-12-21 02:16:27.400257: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-21 02:16:27.421403: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-21 02:16:27.427240: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-21 02:16:28.720753: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
usage: tensorboard [-h] [--helpfull] [--logdir PATH] [--logdir_spec PATH_SPEC] [--host ADDR]
                   [--bind_all] [--port PORT] [--reuse_port BOOL] [--load_f

In [51]:
from object_detection.utils.visualization_utils import visualize_boxes_and_labels_on_image_array

# Load model hasil pelatihan
detection_model = tf.saved_model.load('/content/drive/My Drive/faster_rcnn_model/saved_model')

# Visualisasi
image_np = ...  # Load gambar validasi
output_dict = detection_model(image_np)
visualize_boxes_and_labels_on_image_array(
    image_np,
    output_dict['detection_boxes'],
    output_dict['detection_classes'],
    output_dict['detection_scores'],
    category_index
)
plt.imshow(image_np)
plt.show()


OSError: SavedModel file does not exist at: /content/drive/My Drive/faster_rcnn_model/saved_model/{saved_model.pbtxt|saved_model.pb}